# The Distribution of Income and Crime By Residential Subdivision in Cary, North Carolina


*Analysis by Alexander Ng*
for __CUNY SPS, Data Science 608__

__December 2020__

<img src="https://www.townofcary.org/Home/ShowPublishedImage/26839/637414707996500000" width ="800" height=400 align='right' />
<font color="gray">Source: [www.townofcary.org]</font>



## Background

This data application shows how income and crime is distributed across
the residential subdivisions of the Town of Cary in North Carolina.

The intended reader of this application is a new or potential resident of Cary who may be seeking housing in this area and wishes to have quantitative data to understand the many (over 700) residential subdivisions.

When I was house hunting in Cary, I struggled to understand the regional differences and socioeconomic nuances of **residential subdivisions**.   The residential subdivision is the basic unit of local community - much like a neighborhood is a larger urban setting.  After touring 10 houses in different subdivisions, they all start to blend together.

  1.  Residential subdivisions even have limited authority in the form of Homeowner associations (HOA) to define and enforce local community norms.  Residents have to join an HOA and hence adopt the subdivision's bylaws or standards.

  2.  Subdivisions are often planned by real estate developers and built and marketed over a limited period of time.  They tend to have homogenous house prices, design, land allotment, amenities.   Thus, a common cohort of owners tends to buy into newer subdivisions.
  
However, there is limited ability to analyze residential subdivisions through public data sources because they are not units of governmental administration.

This application looks at two basic questions.  

  1.  How is crime density relatively distributed geospatially within residential subdivisions?   
  2.  How is income distributed relatively geospatially over the same space?
  
This information may help add context to visual inspection of these areas.

## Sources

The information used in this application comes from multiple data soures.  

1.  __Subdivision and Town boundary__ data comes from Town of Cary Open Data portal in GEOJSON format
  [Residential Boundaries](https://data.townofcary.org/explore/dataset/httpmapstownofcary0/table/?sort=name)  Note that 4 residential subdivisions were omitted because of missing GIS data from its source.  
  
2.  __Crime Incident__ data comes from the Town of Cary Police department Open Data Portal in GEOJSON format.
[Police Incidents](https://data.townofcary.org/explore/dataset/cpd-incidents/information)   Approximately half of crime incidents in the data set occurred outside of any residential subdivision.
Those crimes are shown on the heatmap but don't affect subdivision relative color coding.  Only crimes within the 2019-Nov 2020 period are included in the analysis due to the rapid growth in housing construction in the Town.

3. __Income Data__ comes from the US Census at the Census Tract level reported from the ACS Survey as of 2018.
  We use median household income of each US Census tract that overlaps the interior of Town of Cary limits.
  The Town spans two counties with the majority in Wake and a small part of the western area spilling into Chatham counties.  This is handled by taking an area weighted average of each residential subdivision with each census tract to estimate household income.  The source for this data is [Census Reporter B19013 table](https://censusreporter.org/data/map/?table=B19013&geo_ids=140|06000US3718390536,140|06000US3718390536,150|06000US3718390536,16000US3710740&primary_geo_id=06000US3718390536#column|B17001002,sumlev|140)
  
## Using the Tool

The tool aggregates crime data based on residential subdivision and shows crime density by area of each subdivision.   We report a metric called the crime density per squared kilometer per year to color the heatmap.  Other metrics are calculated and reported:  crime density is reported by number of incidents per lot per annum.  However, some residential subdivisions count **approved lots** in strange ways.  Please be aware that the crime density metric penalizes apartment buildings, condos and more urban areas compared to single family detached housing.  Ideally, a population based crime density metric would be more useful.  That suggests looking at crime at the census tract level but that may be unhelpful to the potential home buyer.

The second heatmap shows income distribution from census data.  The distribution bins (which define the color scheme) only consider income data for regions abutting Town of Cary.  In fairness, income data should probably be compared to surrounding communities of the Research Triangle area.

## Still A Great Place to Live
 
According to Wikipedia, Cary has a crime rate of 84 violent crimes per 100,000 residents, a rate almost eight times lower than its larger neighbor, Charlotte, North Carolina.  [Cary, Wikipedia](https://en.wikipedia.org/wiki/Cary,_North_Carolina).   Bestplaces.net states that 2020 violent rate for Cary are less than half of the national average (8.9 vs. 22.7).  [Bestplaces.net Crime rates](https://www.bestplaces.net/crime/?city1=53710740&city2=53711800).  
This analysis dissects the Town of Cary from the lens of fear (crime) and greed (income) because these are valid considerations for a homebuyer.  Likewise, household income data for Cary is significantly higher than the national average as well as the Wake County average.  [Census Reporter](https://censusreporter.org/profiles/16000US3710740-cary-nc) shows a median household income of $106,304 nearly double the amount of North Carolina and 30 percent higher than the surrounding Raleigh Metro Area.  

But through the lens of data science, these public data sources can provide insights that speak to our fear (crime) and greed (income).



In [3]:
import folium
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from folium.plugins import MarkerCluster
import IPython
import branca.colormap as cm  # For Folium color maps


In [4]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.mode.use_inf_as_na = True # allows isna() to work for Inf, -Inf

In [5]:
#Diagnostic
#os.getcwd()

In [5]:
#root_dir = "/Users/alexanderng/Documents/final608/"
root_dir = "./"

In [8]:
subdivisions = gpd.read_file(root_dir + "httpmapstownofcary0.geojson")

# Add a synthetic primary key from 1...length because none of the existing fields can serve that role
# -------------------------------------------------------------------------------------------------------
ids = pd.Series(range(1, len(subdivisions)+1))

subdivisions['id'] = ids

# Data Cleaning
# Remove residential subdivisions with undefined geometries.
# Only a few such subdivisions were found.
# -----------------------------------------------------
subdivisions = subdivisions[ ~( subdivisions.geometry.is_empty | subdivisions.geometry.isna())]
subdivisions.crs



<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
#diagnostic
IPython.display.display(subdivisions.tail())

,category,name,shape_stlength,shape_starea,approvedlots,description,geometry,id
712,Single-Family Detached,Oaks at Sears Farm Subdivision (South),6702.561640,2.083824e+06,142,Single-Family Detached,"POLYGON ((-78.87620 35.80276, -78.87620 35.802...",713
713,Townhome,Park West Townes,3799.981979,6.100993e+05,59,Townhome,"POLYGON ((-78.80466 35.80278, -78.80467 35.802...",714
714,Single-Family Detached,Glen at Westhigh,6667.033011,1.872354e+06,89,Single-Family Detached,"POLYGON ((-78.83519 35.76583, -78.83520 35.765...",715
715,Single-Family Detached,Darlington,3300.381310,6.448097e+05,23,Single-Family Detached,"POLYGON ((-78.90740 35.78008, -78.90738 35.778...",716
716,Townhome,Pipers Grove Townhomes,3839.286558,6.296047e+05,55,None,"POLYGON ((-78.74149 35.74253, -78.74073 35.742...",717


In [10]:
scaleFactor = 1.4147856508e-07  # derived from comparing coordinate system of GEOJSON to World Mercator projected CRS for meters distance.
subdivisions['km2'] = subdivisions['shape_starea'] * scaleFactor

#diagnostic
IPython.display.display(subdivisions.head(100))

,category,name,shape_stlength,shape_starea,approvedlots,description,geometry,id,km2
0,Single-Family Detached,Southwick,4109.283619,8.011673e+05,49,Single-Family Detached,"POLYGON ((-78.84659 35.77734, -78.84592 35.777...",1,0.113348
1,Single-Family Detached,Forest Creek,7206.394892,1.847059e+06,92,Single-Family Detached,"POLYGON ((-78.78696 35.71080, -78.78696 35.710...",2,0.261319
2,Single-Family Detached,Stonehaven,5661.139998,8.056796e+05,52,Single-Family Detached,"MULTIPOLYGON (((-78.77150 35.75610, -78.77148 ...",3,0.113986
3,Single-Family Detached,Farmington Woods,12123.627097,4.020845e+06,181,Single-Family Detached,"POLYGON ((-78.78071 35.76445, -78.78065 35.764...",4,0.568863
4,Single-Family Detached,Hunter Park,1399.953258,1.214313e+05,13,Single-Family Detached,"POLYGON ((-78.77593 35.78384, -78.77633 35.783...",5,0.017180
5,Single-Family Detached,Ashworth,2929.635644,2.757870e+05,14,Single-Family Detached,"POLYGON ((-78.77979 35.77576, -78.77813 35.775...",6,0.039018
6,Single-Family Detached,Ridgecrest,666.922875,2.751113e+04,3,Single-Family Detached,"POLYGON ((-78.78984 35.78390, -78.78986 35.783...",7,0.003892
7,Single-Family Detached,Oakwood Heights,8875.000065,4.315995e+06,230,Single-Family Detached,"POLYGON ((-78.80156 35.79028, -78.80155 35.790...",8,0.610621
8,Single-Family Detached,Ridgepath,6281.442460,1.141353e+06,65,Single-Family Detached,"MULTIPOLYGON (((-78.77405 35.75116, -78.77440 ...",9,0.161477
9,Single-Family Detached,Preston Glen,2400.268135,2.989514e+05,19,Single-Family Detached,"POLYGON ((-78.84205 35.80159, -78.84161 35.801...",10,0.042295


In [12]:
#diagnostic
# Load the subdivisions data into memory and display as folium interactive map
cary_folium = folium.Map(location=[35.79374, -78.76901, ], zoom_start=12)

style_func_subdivisions = lambda x: { 'fillColor' : 'cornsilk', 'fillOpacity': 0.2, 'color' : 'gray', 'weight': 1}

folium.GeoJson(subdivisions[['geometry', 'name', 'category', 'approvedlots']], 
               style_function = style_func_subdivisions ,
               tooltip = folium.features.GeoJsonTooltip(fields=['name', 'category' , 'approvedlots'],
                                                        aliases = ['Subdivision', 'category', 'Number Lots'],
                                                        labels = True,
                                                 sticky = True,
                                                 toLocaleString = True
                                                ) ).add_to(cary_folium)


    

In [13]:
#diagnostic
# Subdivisions map
cary_folium

In [14]:
# Load the town boundary provided.
cary_boundary_final608 = gpd.read_file(root_dir + "cary_boundary.geojson")

In [ ]:
#Diagnostic
#cary_boundary_final608.head()

In [ ]:
#Diagnostic map
# Shows the income levels and boundary of the town.
#income_boundary_map = folium.Map(location=[35.79374, -78.76901, ], zoom_start=12)
#folium.GeoJson(cary_boundary_final608.geometry).add_to(income_boundary_map)


In [ ]:
# Load the 2018 ACS Survey of Town of Cary Household Median Income for the relevant
# census tracts overlapping with Town of Cary boundaries.
# -----------------------------------------------------------------------
acs_income = gpd.read_file(root_dir + "acs2018_5yr_B19013_14000US37183053513.geojson")

# Diagnostic
#acs_income.head()

In [ ]:
# Diagnostic
#acs_income.shape

In [ ]:
# Diagnostic
#style_func_acs_income = lambda x: { 'color': 'black', 'fillColor' : '#ccffcc', 'fillOpacity': 0.5, 'color' : 'violet', 'weight': 1}

#folium.GeoJson(acs_income[['geometry', 'name',
#                           'B19013001']], 
#               style_function = style_func_acs_income ,
#               tooltip = folium.features.GeoJsonTooltip(fields=['name', 'B19013001'],
#                                                        aliases = ['Region', 'Median household income'],
#                                                        labels = True,
#                                                 sticky = True,
#                                                 toLocaleString = True
#                                                ) ).add_to(income_boundary_map)




In [ ]:
#Diagnostic
#income_boundary_map

In [ ]:
crimes = gpd.read_file(root_dir +  "cpd-incidents.geojson")

# Clean up the incidents lacking geospatial location data 
# -----------------------------------------------------------
crimes = crimes[ pd.notnull( crimes['geometry'] ) ]

crimes_recent = crimes[ ( crimes['year']=='2019' ) | ( crimes['year'] == '2020') ]

# Diagnostic
#crimes_recent.shape

In [ ]:
# Diagnostic
#IPython.display.display(crimes_recent.head())

In [ ]:
# Diagnostic
#crimes_recent.columns

In [ ]:
# This check the crime data is inside the town boundaries.  It is.
#
#crime_boundary_map = folium.Map(location=[35.79374, -78.76901, ], zoom_start=12)
#folium.GeoJson(cary_boundary_final608.geometry).add_to(crime_boundary_map)
#folium.GeoJson(crimes_recent.geometry).add_to(crime_boundary_map)
#crime_boundary_map

In [ ]:
# Diagnostic
# Figure out how to use MarkerClusters in folium
# --------------------------------------------------------------------------------------
#crime_subdivision_map = folium.Map(location=[35.79374, -78.76901, ], zoom_start=12)
#folium.GeoJson(cary_boundary_final608.geometry).add_to(crime_subdivision_map)

#crime_cluster = MarkerCluster().add_to(crime_subdivision_map)

# Popup Background Color

#bcol = "lightskyblue"

#for incident in range(len(crimes_recent) ):    
#    c = crimes_recent.iloc[incident]
#    g = c.geometry
#    location = [ g.y, g.x]
    
#    details_fmt = """<body style=\"background-color:{0};\">
#                   <p>Category: {1.crime_category}</p>
#                   <p>Date: {1.newdate}</p>
#                   <p>Detail: {1.crime_type}</p>
#                   <p>Beat: {1.beat_number}</p>
#                   <p>Subdivision: {1.residential_subdivision}</p>
#                   </body>""".format(bcol, c)
    
#    t = folium.IFrame(details_fmt, width=300, height = 200)
#    popup = folium.Popup(t)
#    folium.Marker( location = location,
#                   popup = popup).add_to(crime_cluster)
                    
#folium.GeoJson(subdivisions[['geometry', 'name', 'category', 'approvedlots']], 
#               style_function = style_func_subdivisions ,
#               tooltip = folium.features.GeoJsonTooltip(fields=['name', 'category' , 'approvedlots'],
#                                                        aliases = ['Subdivision', 'category', 'Number Lots'],
#                                                        labels = True,
#                                                 sticky = True,
#                                                 toLocaleString = True
#                                                ) ).add_to(crime_subdivision_map)

#crime_subdivision_map

In [ ]:
#  Spatial join is needed to identify which crimes fall inside a residential subdivision
#  We uses the geopandas spatial join
#
j_crimes_subdivision = gpd.sjoin(crimes_recent, subdivisions, how='inner', op='within')

# Diagnostics
#j_crimes_subdivision.shape

In [ ]:
# Diagnostics
#IPython.display.display(j_crimes_subdivision.tail())

In [ ]:
# Define a subset of required columns and drop the rest.
# Note that we don't use any primary key for the crime data since it is
# treated only at the statistical summary level except when plotting the point data.
#
jcs = j_crimes_subdivision.loc[:, ['id_left', 'year', 'crime_type', 
                                   'crime_category', 'newdate' , 'geometry', 
                                   'name', 'id_right'] ]
#jcs.head()


In [ ]:
# Aggregate crime statistics by subdivision (id_right) and define the tot_crime column
# This summarizes the total count of crime incidents per residential subdivision
# over the entire data set from 2019 to Nov 30, 2020.
# Change the groupby output from Series to DataFrame.
# --------------------------------------------------------------------------------------
sub_crime_total = jcs.groupby(['id_right', 'name']).size().to_frame(name='tot_crime')

sub_crime_total.reset_index(inplace=True)  # flatten the hierarchical index to simple columns

# Rename the aggregated column but keep the other column names.
sub_crime_total.columns = ['id_right', 'subdivision', 'tot_crime']

# Resulting dataframe has 3 columns:  id_right (of the subdivision), subdivision (name), 
#    tot_crime (total crimes incidents)
# Subdivisions with no crimes are not represented.

# Diagnostic
# ------------------------------------------------------------------------------------------------------------------
#IPython.display.display(sub_crime_total.sort_values(by='tot_crime', ascending=False))



In [ ]:
# Data cleaning step: resolved by adding an integer key to the subdivisions dataframe
# identify and remove the duplicated residential subdivisions.
#x = subdivisions[ subdivisions.duplicated( subset=['name'], keep=False)][['name','geometry', 'approvedlots','category']].sort_values(by='name')
#IPython.display.display(x)


In [ ]:
# Diagnostic
#  Investigated when some subdivision names are repeated.   I.e. Subdivision names are not a primary key
#  because some areas had multiple phases of development.  This affected a small fraction (about 24 of 713 developments)
#  None of them had to be removed due to a geospatial inspection of boundaries.
#  Often the two repeated subdivision names were adjacent polygons.
# ------------------------------------------------------------------------
#duplicate_subdiv_map = folium.Map(location=[35.79374, -78.76901, ], zoom_start=12)

#style_func_subdivisions = lambda x: { 'fillColor' : 'red', 'fillOpacity': 0.5, 'color' : 'gray', 'weight': 1}

#folium.GeoJson(x[['geometry', 'name', 'category', 'approvedlots']], 
#               style_function = style_func_subdivisions ,
#               tooltip = folium.features.GeoJsonTooltip(fields=['name', 'category' , 'approvedlots'],
#                                                        aliases = ['Subdivision', 'category', 'Number Lots'],
#                                                        labels = True,
#                                                 sticky = True,
#                                                 toLocaleString = True
#                                                ) ).add_to(duplicate_subdiv_map)
#display(duplicate_subdiv_map)


In [ ]:
# Diagnostic
#crimes.id.unique
#crimes.id.count()

In [ ]:
# Spatial Join of the Subdivision data with the Census Tract data which has the ACS median income
# data.  Note that a subdivision may span 1 or 2 census tracts.
# -------------------------------------------------------------------------------
j_subdiv_census = gpd.sjoin(subdivisions, acs_income, op='intersects')


# Diagnostic
#j_subdiv_census.shape

In [ ]:
# Diagnostic
#acs_income.crs

In [ ]:
# Diagnostic
#subdivisions.head()

In [ ]:
# Diagnostic
# Investigate which subdivisions cross 2 or more census tracts.
# We need to get the area-weighted average median income for the subdivision.
subdivs_multi_census = j_subdiv_census[ j_subdiv_census.duplicated( subset=['id'], keep=False)]

#IPython.display.display(subdivs_multi_census.sort_values(by='name_left').name_left.unique() )

In [ ]:
# Overlay allows us to partition the subdivision into the disjoint union of areas
# with distinct census tracts.  Just take the weighted average median income using the
# overlay areas as the primitive components.
# --------------------------------------------------------------------------------
overlay_subdiv_acs = gpd.overlay( subdivisions, acs_income, how='intersection')

overlay_subdiv_copy = gpd.GeoDataFrame(overlay_subdiv_acs)

overlay_subdiv_copy = overlay_subdiv_copy.to_crs(epsg=3857)

overlay_subdiv_acs['area_crs'] = overlay_subdiv_copy.geometry.area/1000000.0

In [ ]:
# IPython.display.display(overlay_subdiv_acs)

In [ ]:
#Debug
#IPython.display.display(overlay_subdiv_acs.sort_values(by=['name_1', 'id']))

In [ ]:
#
#  Grouped Weighted Average function allows us to calculate
#  an census tract weighted median household income for each 
#  residential subdivision.
# --------------------------------------------------------------------
grouped = overlay_subdiv_acs.groupby('id')

def wavg(group):
    d = group['B19013001']
    w = group['area_crs']
    return ( d * w).sum() / w.sum()

subdiv_wavg_income = grouped.apply(wavg)

# Constructs a dataframe of the area-weighted average median household income per subdivision.
# We need to join this back to the original subdivision dataset.
subdiv_wavg_income_df = pd.DataFrame({ 'id': subdiv_wavg_income.index, 'income': subdiv_wavg_income.values})

# Diagnostics
#IPython.display.display(subdiv_wavg_income_df)

In [ ]:
#  The next lines:  pick the row of the dataframe with largest area_crs within each
#   group and select the census tract (name_2) associated with that row.
# --------------------------------------------------------------------------
idx = grouped['area_crs'].transform(max) == overlay_subdiv_acs['area_crs']

primary_census_tract = overlay_subdiv_acs[idx].sort_values(by='name_1')[['name_1', 'id', 'name_2']]

# Diagnostics
#IPython.display.display(primary_census_tract)


In [ ]:
# Diagnostic
#
# Create a folium map to check if the census tracts overlay the residential subdivisions
# completely.  No way to do this by looking at numeric data -- must be visual.
#
#overlay_folium = folium.Map(location=[35.79374, -78.76901, ], zoom_start=12)

#style_func_subdiv_overlay = lambda x: { 'fillColor' : 'blue', 
#                                       'fillOpacity': 0.3, 'color' : 'gray', 'weight': 1}

#folium.GeoJson(acs_income).add_to(overlay_folium)

#folium.GeoJson(overlay_subdiv_acs[['geometry', 'name_1', 'category', 'approvedlots', 'name_2', 'B19013001', 'area_crs']], 
#               style_function = style_func_subdiv_overlay ,
#               tooltip = folium.features.GeoJsonTooltip(fields=['name_1', 'category' , 'approvedlots',
#                                                                'name_2', 'B19013001', 'area_crs'],
#                                                        aliases = ['Subdivision', 'category', 'Number Lots', 
#                                                                   'Census Tract', 'Household Income', 'km2'],
#                                                        labels = True,
#                                                 sticky = True,
#                                                 toLocaleString = True
#                                                ) ).add_to(overlay_folium)



#overlay_folium

In [ ]:
# Merge the household income back to the subdivision data.
#
subdivision_rpt = subdivisions.merge( subdiv_wavg_income_df, left_on = 'id', right_on='id')

subdivision_rpt = subdivision_rpt.merge(primary_census_tract, left_on = 'id', right_on = 'id')

# Diagnostic
#print(subdivision_rpt.shape)
#IPython.display.display(subdivision_rpt)
#type(subdivision_rpt[['id', 'category', 'name', 'approvedlots', 'geometry', 'km2', 'income', 'name_2']])

In [ ]:
#
#  Many subdivisions experienced no crimes - so the left join of crime totals will have Null rows.
#  We fill them in with zeros.
yy = subdivision_rpt.merge(sub_crime_total, how='left', left_on = 'id', right_on = 'id_right' )
yy['tot_crime'].fillna(0, inplace=True)

# Diagnostic
#print(yy.shape)
# Diagnostic
#IPython.display.display(yy.head(20))

# Define formulas to calculate 
# The crime density by lot (i.e. a proxy for crime density per capita)
# But we only have data on number of approved lots -- not on population.
#
def crime_lot_calc(row ):
    t = row['tot_crime']
    lots = row['approvedlots']
    
    if t > 0:
        if lots > 0:
            return t / lots * (12/23) # Annualizing crime rate per annum
        else:
            return t / 1.0 * (12/23) # Treat entire subdivision as 1 lot
    else:
        return 0

# Returns years between crime incidents per lot given
# the rate of crime in the subdivision and number of lots
# assuming each lot's experience is equal within its subdivision
# IF zero crimes are detected in the sample period
# return np.NaN
# -----------------------------------------------------
def crime_years_calc(row):
    t = row['tot_crime']
    if t > 0:
        return 1.0/ crime_lot_calc(row)
    else:
        return np.nan

# Define the columns
yy['r_crime_lots'] = yy.apply( lambda row: crime_lot_calc(row), axis = 1)
yy['r_years_crime_lots'] = yy.apply(lambda row: crime_years_calc(row), axis = 1)

# Diagnostic
#IPython.display.display(yy.head() ) 

In [ ]:
# Calculate the Crime Density
#
yy['r_crime_km2'] = yy['tot_crime'] / yy['km2'] * (12/23)  # Annualizing the crime rate per annum over 23 months of data

# Diagnostic
#IPython.display.display(yy[ ( pd.isna( yy['r_crime_km2'] ) )| ( yy['km2']==0 ) ] ) 

yy.rename(columns={ 'name_2' : 'census_tract'}, inplace=True)


ya = yy[['name', 'category', 'id', 'approvedlots', 'income', 
         'census_tract', 'description' ,'km2', 'tot_crime' ,
         'r_crime_km2', 'r_crime_lots', 'r_years_crime_lots' , 'geometry']]

# Diagnostic
#
#IPython.display.display(ya.sort_values(by='r_crime_km2', ascending=False))


In [ ]:
#subdivisions[ (subdivisions['category']!='Single-Family Detached') & ( subdivisions['approvedlots'] < 5 )]

In [ ]:
# Diagnostics
#df = pd.DataFrame({'x': ya.income, 'y': ya.tot_crime, 'z' : ya.r_crime_km2 , 'w': ya.r_years_crime_lots})
#ax1 = df.plot.scatter(x='x', y='y')
#ax2 = df.plot.scatter(x='x', y='z')
#ax3 = df.plot.scatter(x='x', y='w')

## Visualizing Crime Density and Incidents

The map below shows 3 layers:  Town of Cary boundary, crime incident points and residential subdivisions.
These can be toggled on or off.   The viewer can pan and zoom to see subdivision boundaries, crime statistics (in the tooltip) and incident details.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

heatmap_crime_subdiv = folium.Map(location=[35.79374, -78.76901, ], zoom_start=13,
                                 width= '100%', height = '100%')



folium.GeoJson(cary_boundary_final608.geometry,
              name="Town Boundary",
              style_function = lambda x: {"weight": 0.5 , 'color':  'yellow', 'fillColor': 'purple', 'fillOpacity': 0.3 } 
              ).add_to(heatmap_crime_subdiv)


crime_metric =  'r_crime_km2'

linearColorMap = cm.LinearColormap(["skyblue", "green", "yellow", "red"])

linearColorMap = linearColorMap.to_step(data=ya[crime_metric], method = 'quant',  quantiles=[0, 0.05, 0.1, 0.25, 0.4, 0.5, 0.6, 0.75, 0.9, 0.95, 0.99, 1])

linearColorMap.caption = 'Crime Density by Incidents/sq Km / Year' 

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':linearColorMap(x['properties'][crime_metric]), 
                            'fillOpacity':0.8}

highlight_function = lambda x: {'fillColor': 'green', 
                                'color':'yellow', 
                                'fillOpacity': 0.30, 
                                'weight': 0.1}


crsubdiv1 = folium.features.GeoJson(
        ya,
         name="Crime Density by Subdivision" ,
        style_function=style_function,
        control=True,
        highlight_function=highlight_function,
        tooltip=folium.features.GeoJsonTooltip(fields=['name', 
                                                       'category',
                                                       'km2',
                                                       'income',
                                                       'census_tract' ,
                                                       'tot_crime' ,
                                                       'r_crime_km2' ,
                                                       'r_years_crime_lots' ,
                                                       'approvedlots',
                                                       'description'
                                                      ],                         
            aliases=[ 'Subdivision:',  
                      'Residential Category:',
                      'Area (km2):',
                      'Household Median Income:' ,
                      'Census Tract:' ,
                      '# Crime Incidents 2019-2020:' ,
                      'Crimes/km2 / year:' ,
                      'Avg Years to Incident per Lot:' ,
                      'Num Lots:' ,
                      'Subdivision Details:'
                    ],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
            sticky=True
                                              )
    )
linearColorMap.add_to(heatmap_crime_subdiv)
heatmap_crime_subdiv.add_child(crsubdiv1)

#
# Add the crime incident data into the heat map as an optional layer.
# -----------------------------------------------------------------------

bcol = 'goldenrod'
featureGroup = folium.FeatureGroup(name='Crime Incidents', show = False)

heatmap_crime_subdiv.add_child(featureGroup)

crime_cluster = MarkerCluster().add_to(featureGroup)

for incident in range(len(crimes_recent) ):
    
    c = crimes_recent.iloc[incident]
    g = c.geometry
    location = [ g.y, g.x]
    
    details_fmt = """<body style=\"background-color:{0};\">
                   <p>Category: {1.crime_category}</p>
                   <p>Date: {1.newdate}</p>
                   <p>Detail: {1.crime_type}</p>
                   </body>""".format(bcol, c)
    
    t = folium.IFrame(details_fmt, width=300, height = 200)
    popup = folium.Popup(t)
    folium.Marker( location = location,
                   popup = popup).add_to(crime_cluster)


folium.LayerControl().add_to(heatmap_crime_subdiv)

# Town of Cary - Crime Density within Residential Subdivisions (2019-2020)

heatmap_crime_subdiv


## Visualizing Median Income Distribution

At the residential subdivision level, median household income is presented based on the US Census ACS survey results from 2018.
The overlay between census tracts and subdivisions is used to weight the income by the areas by which census tracts partition each subdivision.


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

heatmap_income_subdiv = folium.Map(location=[35.79374, -78.76901, ], zoom_start=13,
                                 width= '100%', height = '100%')



folium.GeoJson(cary_boundary_final608.geometry,
              name="Town Boundary",
              style_function = lambda x: {"weight": 0.5 , 'color':  'yellow', 'fillColor': 'purple', 'fillOpacity': 0.3 } 
              ).add_to(heatmap_income_subdiv)


metric =  'income'

incomeColorMap = cm.LinearColormap(["red", "tomato", "yellow", "green", "blue"])

incomeColorMap = incomeColorMap.to_step(data=ya[metric], method = 'quant',  quantiles=[0, 0.02, 0.05, 0.1, 0.25, 0.4, 0.5, 0.6, 0.75, 0.9, 0.95, 0.98, 1])

incomeColorMap.caption = 'Household Median Income Weighted by Area' 

income_style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':incomeColorMap(x['properties'][metric]), 
                            'fillOpacity':0.8}

income_highlight_function = lambda x: {'fillColor': 'green', 
                                'color':'yellow', 
                                'fillOpacity': 0.30, 
                                'weight': 0.1}


income_subdiv1 = folium.features.GeoJson(
        ya,
         name="Household Income by Subdivision" ,
        style_function=income_style_function,
        control=True,
        highlight_function=income_highlight_function,
        tooltip=folium.features.GeoJsonTooltip(fields=['name', 
                                                       'category',
                                                       'km2',
                                                       'income',
                                                       'census_tract' ,
                                                       'tot_crime' ,
                                                       'r_crime_km2' ,
                                                       'r_years_crime_lots' ,
                                                       'approvedlots',
                                                       'description'
                                                      ],                         
            aliases=[ 'Subdivision:',  
                      'Residential Category:',
                      'Area (km2):',
                      'Household Median Income:' ,
                      'Census Tract:' ,
                      '# Crime Incidents 2019-2020:' ,
                      'Crimes/km2 / year:' ,
                      'Avg Years to Incident per Lot:' ,
                      'Num Lots:' ,
                      'Subdivision Details:'
                    ],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
            sticky=True
                                              )
    )
incomeColorMap.add_to(heatmap_income_subdiv)
heatmap_income_subdiv.add_child(income_subdiv1)


folium.LayerControl().add_to(heatmap_income_subdiv)


# Town of Cary - Income within Residential Subdivisions (2019-2020)
heatmap_income_subdiv
